In [1]:
import os
import logging
from tqdm import tqdm

import utils
import config
import datasets
import validate

import numpy as np
import torch
import torchmetrics

In [2]:
import onnx
import onnxruntime

# Logging for Evaluation Reports

In [14]:
log_path = './eval_reports_log/no_comp.log'

logger = logging.getLogger("GonLogger")
logger.propagate = False
logger.setLevel(logging.INFO)
file_handler = logging.FileHandler(log_path)
formatter = logging.Formatter('%(message)s')
file_handler.setFormatter(formatter)

# add file handler to logger
logger.addHandler(file_handler)

logger.info('No Compression.\n' +  
            #'\tConv341 defined as big layer.\n' +
            f'\tWeights bit width = {config.NO_COMP_WEIGHTS_BIT_WIDTH}.\n' +
            f'\tBig Layers Weights bit width = {config.NO_COMP_BIG_LAYERS_WEIGHTS_BIT_WIDTH}.\n' +
            f'\tActivations bit width = {config.NO_COMP_ACTIVATIONS_BIT_WIDTH}.\n' +
            f'\tBias bit width = {config.NO_COMP_BIAS_BIT_WIDTH}.\n')

# logger.info('Medium Compression.\n' +  
#             #'\tConv341 defined as big layer.\n' +
#             f'\tWeights bit width = {config.MED_COMP_WEIGHTS_BIT_WIDTH}.\n' +
#             f'\tBig Layers Weights bit width = {config.MED_COMP_BIG_LAYERS_WEIGHTS_BIT_WIDTH}.\n' +
#             f'\tActivations bit width = {config.MED_COMP_ACTIVATIONS_BIT_WIDTH}.\n' +
#             f'\tBias bit width = {config.MED_COMP_BIAS_BIT_WIDTH}.\n')

# Load Model

In [15]:
model_path = './models/onnx_fasdd/fassd_no_comp_epoch=59.onnx'
#model_path = './models/onnx_fasdd/medium_fassd_best_f1_mean_epoch=37.onnx'
#model_path = './models/onnx_fasdd/medium_fassd__conv341_big__epoch=93.onnx'

logger.info(f'Using model: {model_path}')

# Datasets

In [6]:
dfire_loader = datasets.get_dfire_val_loader(shuffle=False)
fasdd_uav_loader = datasets.get_fasdd_uav_val_loader(shuffle=False)
fasdd_cv_loader = datasets.get_fasdd_cv_val_loader(shuffle=False)
val_loader = datasets.get_val_loader(shuffle=False)


TEST DFire dataset
DFire Removed wrong images: 0
DFire empty images: 2005
DFire only smoke images: 1186
DFire only fire images: 220
DFire smoke and fire images: 895

Test dataset len: 4306

TEST FASDD UAV dataset
DFire Removed wrong images: 0
DFire empty images: 1997
DFire only smoke images: 846
DFire only fire images: 35
DFire smoke and fire images: 1303

Test FASDD UAV dataset len: 4181

TEST FASDD CV dataset
DFire Removed wrong images: 0
DFire empty images: 6533
DFire only smoke images: 3902
DFire only fire images: 2091
DFire smoke and fire images: 3358

Test FASDD CV dataset len: 15884

TEST DFire dataset
DFire Removed wrong images: 0
DFire empty images: 2005
DFire only smoke images: 1186
DFire only fire images: 220
DFire smoke and fire images: 895

Test dataset len: 4306

TEST FASDD UAV dataset
DFire Removed wrong images: 0
DFire empty images: 1997
DFire only smoke images: 846
DFire only fire images: 35
DFire smoke and fire images: 1303

Test FASDD UAV dataset len: 4181

TEST FAS

# Evaluate ONNX with F1 Mean, Smoke and Fire

In [7]:
precision_metric = torchmetrics.classification.MultilabelPrecision(num_labels = config.N_CLASSES, 
                                                                   threshold = 0.5, 
                                                                   average = None).to('cpu')
recall_metric = torchmetrics.classification.MultilabelRecall(num_labels = config.N_CLASSES, 
                                                             threshold = 0.5, 
                                                             average = None).to('cpu')
accuracy_metric = torchmetrics.classification.MultilabelAccuracy(num_labels = config.N_CLASSES, 
                                                                 threshold = 0.5, 
                                                                 average = None).to('cpu')
f1_metric = torchmetrics.classification.MultilabelF1Score(num_labels = config.N_CLASSES, 
                                                          threshold = 0.5, 
                                                          average = None).to('cpu')

f1_metric_mean = torchmetrics.classification.MultilabelF1Score(num_labels = config.N_CLASSES, 
                                                               threshold = 0.5, 
                                                               average = 'macro').to('cpu')

In [8]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [9]:
def eval_onnx(loader, model_name):

    ort_session = onnxruntime.InferenceSession(model_name, providers=["CPUExecutionProvider"])

    precision_metric.reset()
    recall_metric.reset()
    accuracy_metric.reset()
    f1_metric.reset()
    f1_metric_mean.reset()
    
    loop = tqdm(loader, desc='Validating', leave=True)

    for batch_idx, (img, label) in enumerate(loop):

        for idx in range(config.BATCH_SIZE):
            
            ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img[idx].unsqueeze(dim=0))}
            yhat = ort_session.run(None, ort_inputs)
            yhat = np.array(yhat)
            #yhat = torch.tensor(yhat).squeeze(dim=0)
            yhat = torch.sigmoid(torch.tensor(yhat).squeeze(dim=0))
            target = label[idx].unsqueeze(dim=0)

            precision_metric.update(yhat, target)
            recall_metric.update(yhat, target)
            accuracy_metric.update(yhat, target)
            f1_metric.update(yhat, target)
            f1_metric_mean.update(yhat, target)
    
    precision = precision_metric.compute()
    recall = recall_metric.compute()
    accuracy = accuracy_metric.compute()
    f1 = f1_metric.compute()
    f1_mean = f1_metric_mean.compute()

    precision_metric.reset()
    recall_metric.reset()
    accuracy_metric.reset()
    f1_metric.reset()
    f1_metric_mean.reset()

    print(f'SMOKE -> Precision: {precision[0]:.4f} - Recall: {recall[0]:.4f} - Accuracy: {accuracy[0]:.4f} - F1: {f1[0]:.4f}')
    print(f'FIRE -> Precision: {precision[1]:.4f} - Recall: {recall[1]:.4f} - Accuracy: {accuracy[1]:.4f} - F1: {f1[1]:.4f}')
    print(f'Mean F1 Score: {f1_mean.item():.4f}')
    
    logger.info(f'SMOKE -> Precision: {precision[0]:.4f} - Recall: {recall[0]:.4f} - Accuracy: {accuracy[0]:.4f} - F1: {f1[0]:.4f}')
    logger.info(f'FIRE -> Precision: {precision[1]:.4f} - Recall: {recall[1]:.4f} - Accuracy: {accuracy[1]:.4f} - F1: {f1[1]:.4f}')
    logger.info(f'Mean F1 Score: {f1_mean.item():.4f}')
    
    return (
        {
        'Accuracy': [accuracy[0].item(), accuracy[1].item()],
        'Precision': [precision[0].item(), precision[1].item()],
        'Recall': [recall[0].item(), recall[1].item()],
        'F1': [f1[0].item(), f1[1].item()],
        'F1 mean': f1_mean.item(),
        }
    )

# Models Trained with FASDD Evaluated with FASDD UAV, CV and DFire, together and separated

In [16]:
print("\n________________________________ DFire + FASDD ________________________________")
logger.info("\n________________________________ DFire + FASDD ________________________________")
metrics_dfire_fasdd = eval_onnx(val_loader, model_path)
#print_metrics(metrics_dfire_fasdd)

print("\n________________________________  DFIRE ________________________________")
logger.info("\n________________________________  DFIRE ________________________________")
metrics_dfire = eval_onnx(dfire_loader, model_path)

print("\n________________________________ FASDD UAV ________________________________")
logger.info("\n________________________________ FASDD UAV ________________________________")
metrics_fasdd_uav = eval_onnx(fasdd_uav_loader, model_path)

print("\n________________________________ FASDD CV ________________________________")
logger.info("\n________________________________ FASDD CV ________________________________")
metrics_fasdd_cv = eval_onnx(fasdd_cv_loader, model_path)


________________________________ DFire + FASDD ________________________________


Validating: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [01:19<00:00,  4.77it/s]


SMOKE -> Precision: 0.9217 - Recall: 0.8882 - Accuracy: 0.9117 - F1: 0.9046
FIRE -> Precision: 0.9261 - Recall: 0.9555 - Accuracy: 0.9609 - F1: 0.9406
Mean F1 Score: 0.9226

________________________________  DFIRE ________________________________


Validating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:13<00:00,  5.10it/s]


SMOKE -> Precision: 0.9005 - Recall: 0.8997 - Accuracy: 0.9039 - F1: 0.9001
FIRE -> Precision: 0.9514 - Recall: 0.9180 - Accuracy: 0.9667 - F1: 0.9344
Mean F1 Score: 0.9173

________________________________ FASDD UAV ________________________________


Validating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 65/65 [00:20<00:00,  3.24it/s]


SMOKE -> Precision: 0.9927 - Recall: 0.8905 - Accuracy: 0.9406 - F1: 0.9388
FIRE -> Precision: 0.9694 - Recall: 0.9701 - Accuracy: 0.9805 - F1: 0.9697
Mean F1 Score: 0.9543

________________________________ FASDD CV ________________________________


Validating: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 248/248 [00:47<00:00,  5.23it/s]

SMOKE -> Precision: 0.9084 - Recall: 0.8847 - Accuracy: 0.9066 - F1: 0.8964
FIRE -> Precision: 0.9112 - Recall: 0.9598 - Accuracy: 0.9541 - F1: 0.9348
Mean F1 Score: 0.9156
